In [ ]:
!pip install segmentation-models-pytorch

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, random_split, DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from torchvision.transforms import ToTensor
from PIL import Image
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
from torchvision import transforms
from torchinfo import summary
import timm

In [ ]:
!nvidia-smi -L

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import segmentation_models_pytorch as smp
from torchinfo import summary
import matplotlib.pyplot as plt
from tqdm import tqdm
import wandb

# Constants
NUM_CLASSES = 3
IMAGE_SIZE = (256, 256)
LEARNING_RATE = 0.0001
BATCH_SIZE = 4
NUM_EPOCHS = 50

# Data paths
TRAIN_PATH = '/kaggle/input/bkai-igh-neopolyp/train/train'
TRAIN_MASK_PATH = '/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt'

color_dict = {
    0: [0, 0, 0],  # Background
    1: [255, 0, 0],  # Class 1 (Red)
    2: [0, 255, 0],  # Class 2 (Green)
    # Add more classes if necessary
}
# Utility Functions

def mask_to_rgb(mask, color_dict):
    """Converts mask to RGB image."""
    output = np.zeros((mask.shape[0], mask.shape[1], 3))

    for k in color_dict.keys():
        output[mask==k] = color_dict[k]

    return np.uint8(output)

def save_best_model(epoch, model, optimizer, loss, save_path):
    """Saves the best model."""
    checkpoint = {
        'epoch': epoch,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(checkpoint, save_path)

def get_device():
    """Returns the appropriate device (GPU or CPU)."""
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset Classes

class UnetImageDataset(Dataset):
    """Custom dataset for images and labels."""
    def __init__(self, img_dir, label_dir, resize=None, transform=None):
        """
        Args:
            img_dir (str): Directory containing input images.
            label_dir (str): Directory containing corresponding label masks.
            resize (tuple): Desired image size (height, width).
            transform (callable): Optional transform to be applied to the image.
        """
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.resize = resize
        self.transform = transform
        self.images = os.listdir(self.img_dir)

    def __len__(self):
        """Returns the number of images in the dataset."""
        return len(self.images)

    def read_mask(self, mask_path):
        """Reads and processes the mask image."""
        image = cv2.imread(mask_path)
        image = cv2.resize(image, self.resize)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        lower1 = np.array([0, 100, 20])
        upper1 = np.array([10, 255, 255])

        lower2 = np.array([160,100,20])
        upper2 = np.array([179,255,255])
        lower_mask = cv2.inRange(image, lower1, upper1)
        upper_mask = cv2.inRange(image, lower2, upper2)
        
        red_mask = lower_mask + upper_mask;
        red_mask[red_mask != 0] = 1

        green_mask = cv2.inRange(image, (36, 25, 25), (70, 255, 255))
        green_mask[green_mask != 0] = 2

        full_mask = cv2.bitwise_or(red_mask, green_mask)
        full_mask = np.expand_dims(full_mask, axis=-1) 
        full_mask = full_mask.astype(np.uint8)
        
        return full_mask

    def __getitem__(self, idx):
        """Gets an image and its corresponding label at the given index."""
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx])

        # Read and preprocess the image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, self.resize)

        # Read and preprocess the label
        label = self.read_mask(label_path)

        if self.transform:
            image = self.transform(image)

        return image, label
        
    def show_image(self, idx):
        """Displays the original image and its label."""
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx])

        image = plt.imread(img_path)
        label = plt.imread(label_path)

        fig, axs = plt.subplots(1, 2, figsize=(10, 5))
        axs[0].imshow(image)
        axs[0].set_title('Image')
        axs[1].imshow(label)
        axs[1].set_title('Label')
        plt.show()

# Extended Dataset Class

class UnetDataset(UnetImageDataset):
    """Extended dataset class."""
    def __init__(self, data, targets, transform=None):
        """
        Args:
            data (list): List of input images.
            targets (list): List of corresponding label masks.
            transform (callable): Optional transform to be applied to the image.
        """
        self.data = data
        self.targets = targets
        self.transform = transform

    def __getitem__(self, index):
        """Gets an image and its corresponding label at the given index."""
        image = self.data[index]
        label = self.targets[index]

        if self.transform:
            transformed = self.transform(image=image, mask=label)
            image = transformed['image'].float()
            label = transformed['mask'].float()
            label = label.permute(2, 0, 1)

        return image, label

    def __len__(self):
        """Returns the number of images in the dataset."""
        return len(self.data)

# Data Augmentation

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomGamma(gamma_limit=(70, 130), eps=None, always_apply=False, p=0.2),
    A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# Dataset Split and Creation

image_path = [os.path.join(root, file) for root, _, files in os.walk(TRAIN_PATH) for file in files]
mask_path = [os.path.join(root, file) for root, _, files in os.walk(TRAIN_MASK_PATH) for file in files]

dataset = UnetImageDataset(
    img_dir=TRAIN_PATH,
    label_dir=TRAIN_MASK_PATH,
    resize=(256, 256),
    transform=None
)

images_data = []
labels_data = []
for x,y in dataset:
    images_data.append(x)
    labels_data.append(y)
# Model Initialization

model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=NUM_CLASSES
)

# Data Loader Creation

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset = UnetDataset(images_data[:train_size], labels_data[:train_size], transform=train_transform)
val_dataset = UnetDataset(images_data[train_size:], labels_data[train_size:], transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Model Training

device = get_device()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()
best_val_loss = float('inf')

wandb.login(
    # set the wandb project where this run will be logged
#     project= "PolypSegment", 
    key = "b9575849263a9312a73f76d71d270c8751628e10",
)
wandb.init(project='Unet_polyp-Segmentation')


for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss_epoch = 0  # Initialize train_loss_epoch
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        images, labels = images.to(device), labels.to(device)
        labels = labels.squeeze(dim=1).long()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        train_loss_epoch += loss.item()  # Accumulate training loss for the epoch

    model.eval()
    with torch.no_grad():
        val_loss = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.squeeze(dim=1).long()
            outputs = model(images)
            val_loss += criterion(outputs.float(), labels.long()).item()

    avg_train_loss = train_loss_epoch / len(train_loader)  # Calculate average training loss
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Train Loss: {avg_train_loss:.10f}, Valid Loss: {avg_val_loss:.10f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        save_best_model(epoch, model, optimizer, val_loss, 'colorization_model.pth')
        print('Save new model')
    wandb.log({"Train loss": avg_train_loss, "Valid loss": avg_val_loss})
    
# Model Inference on Test Data

trainsize = 256
model.eval()

for i in os.listdir("/kaggle/input/bkai-igh-neopolyp/test/test"):
    img_path = os.path.join("/kaggle/input/bkai-igh-neopolyp/test/test", i)
    ori_img = cv2.imread(img_path)
    ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
    ori_w = ori_img.shape[0]
    ori_h = ori_img.shape[1]
    img = cv2.resize(ori_img, (trainsize, trainsize))
    transformed = val_transform(image=img)
    input_img = transformed["image"]
    input_img = input_img.unsqueeze(0).to(device)
    with torch.no_grad():
        output_mask = model.forward(input_img).squeeze(0).cpu().numpy().transpose(1, 2, 0)
    mask = cv2.resize(output_mask, (ori_h, ori_w))
    mask = np.argmax(mask, axis=2)
    new_rgb_mask = np.zeros((*mask.shape, 3)).astype(np.uint8)
    mask_rgb = mask_to_rgb(mask, color_dict)
    cv2.imwrite("predicted_mask/{}".format(i), mask_rgb)
    print(+1)


In [ ]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomGamma(gamma_limit=(70, 130), eps=None, always_apply=False, p=0.2),
    A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [ ]:
import os
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

class UNetTestDataClass(Dataset):
    def __init__(self, img_dir, transform=None, target_size=(256, 256)):
        self.img_dir = img_dir
        self.transform = transform
        self.target_size = target_size
        self.images = os.listdir(self.img_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.img_dir, self.images[index])
        pil_image = Image.open(img_path).convert("RGB")

        # Resize image
        pil_image = pil_image.resize(self.target_size, Image.BILINEAR)

        h, w = self.target_size

        # Convert PIL Image to numpy array
        img_array = np.array(pil_image)

        # Apply transformations
        transformed_data = self.transform(image=img_array)
        data = transformed_data["image"] / 255  # Divide by 255 after applying the transformation

        return data, img_path, h, w

In [ ]:
path = '/kaggle/input/bkai-igh-neopolyp/test/test/'
unet_test_dataset = UNetTestDataClass(path, transform)
test_dataloader = DataLoader(unet_test_dataset, batch_size=8, shuffle=True)

In [ ]:
for i, (data, path, h, w) in enumerate(test_dataloader):
    img = data
    break

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

# Assuming that 'img' and 'predict' are your tensors
img = img.cpu()  # Move img tensor to CPU
predict = predict.cpu()  # Move predict tensor to CPU

# Visualization code
fig, arr = plt.subplots(5, 2, figsize=(10, 20))

for i in range(5):
    arr[i][0].imshow(img[i].permute(1, 2, 0).numpy())
    arr[i][1].imshow(F.one_hot(torch.argmax(predict[i], 0)).float().numpy())

plt.show()

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.transforms import Resize, ToPILImage, InterpolationMode
import torch.nn.functional as F

# Assuming 'b' is your input tensor and 'model' is your segmentation model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move the model to the specified device
model = model.to(device)

# Move the input tensor to the same device as the model
b = b.to(device)

# Set the model to evaluation mode
model.eval()

if not os.path.isdir("/kaggle/working/predicted_masks"):
    os.mkdir("/kaggle/working/predicted_masks")
for _, (img, path, H, W) in enumerate(test_dataloader):
    a = path
    b = img
    h = H
    w = W
    
    # Move input tensors to the device
    b = b.to(device)
    h = h.to(device)
    w = w.to(device)

    with torch.no_grad():
        predicted_mask = model(b)
    for i in range(len(a)):
        image_id = a[i].split('/')[-1].split('.')[0]
        filename = image_id + ".png"
        mask2img = Resize((h[i].item(), w[i].item()), interpolation=InterpolationMode.NEAREST)(ToPILImage()(F.one_hot(torch.argmax(predicted_mask[i], 0)).permute(2, 0, 1).float()))
        mask2img.save(os.path.join("/kaggle/working/predicted_masks/", filename))

In [ ]:
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 0] = 255
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    return rle_to_string(rle)

def mask2string(dir):
    ## mask --> string
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = '/kaggle/working/predicted_masks' # change this to the path to your output mask folder
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']
df.to_csv(r'output.csv', index=False)